In [4]:
import threading
import random
import time
from collections import deque

class StockOrderBook:
    def __init__(self):
        self.order_books = {ticker: {"buy": deque(), "sell": deque()} for ticker in range(1, 1025)}
        self.locks = {ticker: threading.Lock() for ticker in range(1, 1025)}

    def addOrder(self, order_type, ticker, quantity, price):
        order = {"quantity": quantity, "price": price}
        with self.locks[ticker]:
            self.order_books[ticker][order_type].append(order)
        print(f"Order added: {order_type.upper()} {ticker} {quantity} @ {price}")
    
    def matchOrder(self, ticker):
        with self.locks[ticker]:
            buy_orders, sell_orders = self.order_books[ticker]["buy"], self.order_books[ticker]["sell"]
            while buy_orders and sell_orders and buy_orders[0]["price"] >= sell_orders[0]["price"]:
                buy, sell = buy_orders[0], sell_orders[0]
                match_qty = min(buy["quantity"], sell["quantity"])
                buy["quantity"] -= match_qty
                sell["quantity"] -= match_qty

                if buy["quantity"] == 0:
                    buy_orders.popleft()
                if sell["quantity"] == 0:
                    sell_orders.popleft()
                    
                print(f"Matched {match_qty} of {ticker}: Buy @ {buy['price']} & Sell @ {sell['price']}")

    # We could change the value of max_transactions to simulate more/less transactions
    # and also the range of quantity and price
    def simulate_transactions(self, max_transactions=20):
        tickers = list(self.order_books.keys())
        for _ in range(max_transactions):
            ticker = random.choice(tickers)
            order_type = random.choice(["buy", "sell"])
            quantity = random.randint(1, 100)
            price = random.randint(50, 200)

            self.addOrder(order_type, ticker, quantity, price)
            self.matchOrder(ticker)
            time.sleep(random.uniform(0.1, 1))

if __name__ == "__main__":
    stock_order_book = StockOrderBook()
    simulation_thread = threading.Thread(target=stock_order_book.simulate_transactions, daemon=True)
    simulation_thread.start()

Order added: BUY 793 83 @ 139


Order added: BUY 104 51 @ 143
Order added: BUY 280 22 @ 116
Order added: BUY 62 52 @ 77
Order added: SELL 495 50 @ 195
Order added: BUY 25 64 @ 144
Order added: SELL 158 57 @ 140
Order added: BUY 271 50 @ 73
Order added: SELL 241 44 @ 104
Order added: BUY 957 46 @ 118
Order added: SELL 724 40 @ 73
Order added: SELL 581 25 @ 198
Order added: SELL 878 89 @ 70
Order added: SELL 899 39 @ 89
Order added: SELL 381 61 @ 168
Order added: SELL 333 49 @ 169
Order added: BUY 766 63 @ 166
Order added: BUY 927 39 @ 189
Order added: SELL 459 71 @ 113
Order added: SELL 50 79 @ 118
